<a href="https://colab.research.google.com/github/marcinmichalsmolen/ZUM/blob/main/INL_MARCIN_SMOLEN_TASK_Poleval2019_Task6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INL PROJEKT Marcin Smoleń s23429 Poleval 2019 Task Automatic cyberbullying detection

POLEVAL 2019 wzor Task 6: Automatic cyberbullying detection http://2019.poleval.pl/index.php/tasks/task6 - Subtask 6.1


Import potrzebnych biliotek


In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import ipykernel
'exec(%matplotlib inline)'
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download("popular")


from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score, precision_score, recall_score)
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.naive_bayes

!pip install stop_words
from stop_words import get_stop_words
stopWords = get_stop_words('pl')

!pip install pystempel
from stempel import StempelStemmer
stemmer = StempelStemmer.polimorf()

import string
from sklearn.utils import shuffle



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=d320ff732c0753950ace67d18da3146aa63a1ae9abeebaac5332929d0743bfca
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop_words
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.8 MB/s eta 0:00:00


Loading: 100%|██████████| 11368252/11368252 [00:09<00:00, 1157772.01bytes/s]


Zdefinijuje funkcje potrzebne do wyczyszczenia setu danych.

W secie mamy ponad 10km tweetow. Wszystkie zaczynaja sie nazwa uzytkownika ktora zostala zanonimizona wiec mozna ja spokojnie usunac.

Usuwamy stopwords, czyli slowa ktore nie maja zadnego znaczenia i nie wnoszą nic do tekstu z punktu widzenia logiki i przekazu. np. a, aby, nigdy, wtedy, wy.

Usuwamy znaki specjalne, czyli !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

gdy juz mamy wyczyszczone dane przeprowadzamy stemming, czyli wydobywamy ze słowa jego podstawę, np. jechac, pojechac, jechalem, pojachalbym
 =zamienia sie na => "jecha"

In [2]:
def removeUserNames(text):
    text['text'] = text['text'].str.replace('@anonymized_account', '')
    return text

def removeStopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stopWords]
    return " ".join(text)

def removeSpecialChars(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def stem(text):
    text = [stemmer.stem(word) for word in text.split()]
    a = [i for i in text if i]
    return " ".join(a)

In [3]:
y_train = pd.read_csv('http://77.55.209.58/inl/task1/training_set_clean_only_tags.txt', delimiter='\t', names=['y'])
X_train = pd.read_csv('http://77.55.209.58/inl/task1/training_set_clean_only_text.txt', delimiter='\t', names=['text'])

In [4]:
len(y_train)

10041

In [5]:
len(X_train)

10041

In [6]:
type(X_train)

pandas.core.frame.DataFrame

In [7]:
X_train = removeUserNames(X_train)
X_train['text'] = X_train['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_train = X_train['text'].values.tolist()
X_train[64]

'wyspa to sezon raz ezyt puchar 12 meczeć i 1 golmójt dużo strzeliłsorrć patr wyspa statystyka'

In [8]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [9]:
X_train[1]

<1x14382 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [10]:
transformer = TfidfTransformer()
X_train = transformer.fit_transform(X_train)

In [11]:
X_train[1]

<1x14382 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

MultinomialNB

In [12]:
model = MultinomialNB()
model.fit(X_train, y_train)

y_predict_train = model.predict(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
print('Acc: ', accuracy_score(y_train,  y_predict_train))

Acc:  0.9168409520964047


In [14]:
print('F1:', f1_score(y_train,  y_predict_train, average = 'weighted'))

F1: 0.8786029464134026


In [15]:
y_test = pd.read_csv('http://77.55.209.58/inl/test1/test_set_clean_only_tags.txt', delimiter='\t', names=['y'])
X_test = pd.read_csv('http://77.55.209.58/inl/test1/test_set_clean_only_text.txt', delimiter='\t', names=['text'])

In [16]:
len(y_test)

1000

In [17]:
len(X_test)

1000

In [18]:
X_test = removeUserNames(X_test)
X_test['text'] = X_test['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_test = X_test['text'].values.tolist()

In [19]:
X_test[1]

'ale mieć szansa zagranie ativus ten proba czysty prowizorka'

In [20]:
X_test = vectorizer.transform(X_test)
X_test[1]

<1x14382 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [21]:
X_test = transformer.transform(X_test)
X_test[1]

<1x14382 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [22]:
y_predict_test = model.predict(X_test)

In [23]:
print('Acc:', accuracy_score(y_test,  y_predict_test))

Acc: 0.866


In [24]:
print('F1:', f1_score(y_test,  y_predict_test, average = 'weighted'))

F1: 0.8038113612004287


Inny model uczenia

In [25]:
model = BernoulliNB()
model.fit(X_train, y_train)

y_predict_train = model.predict(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
print('Acc: ', accuracy_score(y_train,  y_predict_train))

Acc:  0.9242107359824718


In [27]:
print('F1:', f1_score(y_train,  y_predict_train, average = 'weighted'))

F1: 0.8954942598431733


In [28]:
y_predict_test = model.predict(X_test)

In [29]:
print('Acc:', accuracy_score(y_test,  y_predict_test))

Acc: 0.866


In [30]:
print('F1:', f1_score(y_test,  y_predict_test, average = 'weighted'))

F1: 0.8038113612004287



POLEVAL 2019 wzor Task 6: Automatic cyberbullying detection http://2019.poleval.pl/index.php/tasks/task6 - Subtask 6.2

W tym zadaniu uzyje XGBoostClassifier

XGBoost to implementacja algorytmu drzewa gradientowego, który jest powszechnie uznawany ze wzgledu wydajność i dokładność predykcyjną. 

In [31]:
!pip install xgboost
from xgboost import XGBClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
y_train = pd.read_csv('http://77.55.209.58/inl/task2/training_set_clean_only_tags.txt', delimiter='\t', names=['y'])
X_train = pd.read_csv('http://77.55.209.58/inl/task2/training_set_clean_only_text.txt', delimiter='\t', names=['text'])
y_test = pd.read_csv('http://77.55.209.58/inl/test2/test_set_only_tags.txt', delimiter='\t', names=['y'])
X_test = pd.read_csv('http://77.55.209.58/inl/test2/test_set_only_text.txt', delimiter='\t', names=['text'])

In [33]:
X_train = removeUserNames(X_train)
X_train['text'] = X_train['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_train = X_train['text'].values.tolist()
X_train[64]

'wyspa to sezon raz ezyt puchar 12 meczeć i 1 golmójt dużo strzeliłsorrć patr wyspa statystyka'

In [34]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()

X_train = vectorizer.fit_transform(X_train)
X_train = transformer.fit_transform(X_train)

In [35]:
model = XGBClassifier()
model.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(objective='multi:softprob')

In [36]:
y_predict_train = model.predict(X_train)
print('Acc:', accuracy_score(y_train,  y_predict_train))

Acc: 0.9180360521860372


In [37]:
print('F1:', f1_score(y_train,  y_predict_train, average='weighted'))

F1: 0.8816011287626483


In [38]:
X_test = removeUserNames(X_test)
X_test['text'] = X_test['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_test = X_test['text'].values.tolist()

In [39]:
X_test = vectorizer.transform(X_test)
X_test = transformer.transform(X_test)

In [40]:
y_predict_test = model.predict(X_test)
print('Acc:', accuracy_score(y_test,  y_predict_test))

Acc: 0.868


In [41]:
print('F1:', f1_score(y_test, y_predict_test, average='weighted'))

F1: 0.8086017476704171
